<a href="https://colab.research.google.com/github/heispv/bioinformatics/blob/master/lab-of-bioinformatics/CpG_island.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr21.fa.gz

--2024-03-21 15:13:31--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr21.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12709705 (12M) [application/x-gzip]
Saving to: ‘chr21.fa.gz’

chr21.fa.gz         100%[===================>]  12.12M  17.0MB/s    in 0.7s    

2024-03-21 15:13:33 (17.0 MB/s) - ‘chr21.fa.gz’ saved [12709705/12709705]



In [ ]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/database/cpgIslandExt.txt.gz

--2024-03-21 15:13:33--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/database/cpgIslandExt.txt.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 717984 (701K) [application/x-gzip]
Saving to: ‘cpgIslandExt.txt.gz’

cpgIslandExt.txt.gz 100%[===================>] 701.16K  1.98MB/s    in 0.3s    

2024-03-21 15:13:33 (1.98 MB/s) - ‘cpgIslandExt.txt.gz’ saved [717984/717984]



In [ ]:
!gunzip /content/cpgIslandExt.txt.gz

In [ ]:
!gunzip /content/chr21.fa.gz

In [ ]:
input_file_path = '/content/cpgIslandExt.txt'
output_file_path = '/content/chr21.txt'

delimiter = '\t'

with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    for line in input_file:
        columns = line.strip().split(delimiter)
        if len(columns) > 1 and columns[1] == 'chr21':
            output_file.write(line)

In [ ]:
!grep ">" /content/chr21.fa

>chr21


In [ ]:
import numpy as np

In [ ]:
def get_seq(seqfile):
    seq = ''
    with open(seqfile) as f:
        for line in f:
            if line[0] == '>': continue
            seq += line.rstrip()
    return seq

In [ ]:
seq = get_seq("/content/chr21.fa")

In [ ]:
def get_ranges(bedfile, start_col=2, end_col=3):
    ranges = []
    with open(bedfile) as f:
        for line in f:
            line_list = line.split()
            ranges.append((int(line_list[start_col]), int(line_list[end_col])))
    return ranges

In [ ]:
ranges = get_ranges('/content/chr21.txt')

In [ ]:
len(ranges)

446

In [ ]:
ranges

[(5020207, 5023177),
 (5034173, 5034469),
 (5061052, 5061334),
 (5094992, 5095229),
 (5097434, 5097716),
 (5100057, 5100357),
 (5120935, 5121272),
 (5127896, 5128912),
 (5153931, 5154977),
 (5972978, 5973381),
 (6053183, 6053481),
 (6081319, 6081543),
 (6093544, 6094235),
 (6110220, 6112309),
 (6117690, 6118021),
 (6120933, 6121410),
 (6126903, 6127294),
 (6138707, 6139259),
 (6144770, 6145479),
 (6445407, 6445651),
 (6449222, 6449633),
 (6452127, 6452340),
 (6457708, 6458033),
 (6466192, 6468557),
 (6486821, 6487119),
 (6498731, 6499634),
 (6545299, 6545646),
 (6562186, 6562400),
 (6563756, 6563958),
 (6564293, 6565217),
 (6798575, 6798868),
 (6800835, 6801522),
 (6815056, 6815290),
 (6815585, 6816006),
 (6831478, 6832757),
 (6839708, 6840481),
 (6858312, 6859140),
 (6897183, 6897968),
 (7744498, 7745294),
 (7829387, 7830055),
 (8196730, 8197514),
 (8202148, 8202403),
 (8219029, 8221343),
 (8223622, 8224171),
 (8236315, 8236632),
 (8238342, 8238818),
 (8243365, 8243664),
 (8246540, 82

In [ ]:
def get_states(ranges, seq):
    state = np.zeros(len(seq))
    for i, j in ranges:
        state[i:j+1] = 1
    return state[1:]

In [ ]:
def list_to_str(state_list):
    end = ''
    for num in state_list:
        if num:
            end += 'Y'
        else:
            end += 'N'
    return end

In [ ]:
ranges = [(1, 2), (8, 12)]

In [ ]:
state_list = get_states(ranges, 'AAACCCTTTGGGAAAC')
state_list

array([1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.])

In [ ]:
list_to_str(state_list)

'YYNNNNNYYYYYNNN'

In [ ]:
state = get_states(ranges, seq)

In [ ]:
state = list_to_str(state)

In [ ]:
def get_matrices(seq, state, nuc='ACGTN', states='NY'):
    tm_dict = dict()
    ep_dict = dict()
    seq = seq.upper()

    for st1, st2, ch1 in zip(state, state[1:], seq):
        tm_dict[st1+st2] = tm_dict.get(st1+st2, 0) + 1
        ep_dict[st1+ch1] = ep_dict.get(st1+ch1, 0) + 1
    return tm_dict, ep_dict

In [ ]:
from collections import Counter

def get_matrices_optimized(seq, state):
    seq = seq.upper()

    transition_pairs = zip(state, state[1:])
    emission_pairs = zip(state, seq)

    tm_dict = Counter(transition_pairs)
    ep_dict = Counter(emission_pairs)

    tm_dict = {k[0] + k[1]: v for k, v in tm_dict.items()}
    ep_dict = {k[0] + k[1]: v for k, v in ep_dict.items()}

    return tm_dict, ep_dict

In [ ]:
%%time
tm_dict, ep_dict = get_matrices_optimized(seq, state)

CPU times: user 14.9 s, sys: 135 ms, total: 15 s
Wall time: 15.3 s


In [ ]:
%%time
tm_dict, ep_dict = get_matrices(seq, state)

CPU times: user 34.7 s, sys: 138 ms, total: 34.9 s
Wall time: 35.2 s


In [ ]:
import pandas as pd

In [ ]:
expanded_data = [(key[0], key[1], value) for key, value in ep_dict.items()]

df = pd.DataFrame(expanded_data, columns=['Index', 'Column', 'Value'])

pivot_df = df.pivot(index='Index', columns='Column', values='Value')

In [ ]:
pivot_df

Column,A,C,G,N,T
Index,,,,,
N,11769906.0,8063026.0,8106230.0,6621362.0,11800527.0
Y,50758.0,122218.0,120151.0,NaN,55803.0


In [ ]:
normalized_df = pivot_df.div(pivot_df.sum(axis=1), axis=0)
normalized_df

Column,A,C,G,N,T
Index,,,,,
N,0.253875,0.173918,0.174850,0.142822,0.254535
Y,0.145468,0.350265,0.344341,NaN,0.159926
